In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f


ModuleNotFoundError: No module named 'pyspark'

In [3]:
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.databricks:spark-xml_2.12:0.14.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

In [4]:

spark = SparkSession.builder\
    .appName('load-azure')\
    .config(conf=conf)\
    .getOrCreate()


:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
com.microsoft.sqlserver#mssql-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0a662edf-6c95-43d8-97f6-dbb3e4a94ce0;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 473ms :: artifacts dl 27ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central 

In [5]:
person_data = spark.read.csv('s3a://sprint-3-luiza-grupo-09-bucket-brutos/29-05-2022-autismo-csv.csv', header=True)
print(person_data)

22/05/29 17:49:25 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
make_data = spark.read.json('s3a://sprint-3-luiza-grupo-09-bucket-brutos/29-05-2022-autismo-json.json')

print(make_data)

In [ ]:
region_data =  spark.read \
    .format('com.databricks.spark.xml') \
    .options(rowTag='data') \
    .options(rowTag='row') \
    .load('s3a://sprint-3-luiza-grupo-09-bucket-brutos/29-05-2022-autismo-xml.xml')


In [7]:
url = 'jdbc:sqlserver://luiza-grupo-09.database.windows.net:1433;database=autism_data;'
username = 'dbaGrupo09'
password = 'Grupo092022'

In [14]:
person_data.write\
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .mode("overwrite") \
    .option('tableLock', True) \
    .option("url", url) \
    .option("dbtable", 'paciente') \
    .option("user", username) \
    .option("password", password) \
    .save()

In [15]:
make_data.write\
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .mode("overwrite") \
    .option('tableLock', True) \
    .option("url", url) \
    .option("dbtable", 'renda') \
    .option("user", username) \
    .option("password", password) \
    .save()

In [ ]:
region_data.write\
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .mode("overwrite") \
    .option('tableLock', True) \
    .option("url", url) \
    .option("dbtable", 'localidade') \
    .option("user", username) \
    .option("password", password) \
    .save()